# Diabetes Prediction - Final Project CSCI 4050 (ID: Group 16)

## Learning Problem: 
The purpose of this project is to create a Machine Learning model that can predict whether you have diabetes or not and also give you the percentage of confidence that the prediction is accurate. It uses various information including: demographic information (gender, age, number of pregnancies), physical measurements (BMI, blood pressure, skin thickness), lab measurements (blood glucose, HbA1c, insulin) and health history (hypertension, heart disease, smoking status) to help make the predictions. The model was trained using 2 datasets taken from Kaggle and uses a sequence of linear and ReLu layers to help conduct a binary classification of whether a user has diabetes. The model also uses the PyTorch Lightning module and the Adam optimizer to help improve the model's performance.

In [1]:
# Imports
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam

import lightning as L
from lightning.pytorch.callbacks import EarlyStopping
from torchmetrics import Accuracy

from torchinfo import summary

---
## Data Loading and Cleaning

### 1. Loading Raw Data

In [2]:
first_dataset_path  = "data/raw/diabetes.csv"
second_dataset_path = "data/raw/diabetes_prediction_dataset.csv"

first_dataset  = pd.read_csv(first_dataset_path)
second_dataset = pd.read_csv(second_dataset_path)

print("First dataset:", first_dataset.shape)
print("Second dataset:", second_dataset.shape)
first_dataset.head(), second_dataset.head()

First dataset: (768, 9)
Second dataset: (100000, 9)


(   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
 0            6      148             72             35        0  33.6   
 1            1       85             66             29        0  26.6   
 2            8      183             64              0        0  23.3   
 3            1       89             66             23       94  28.1   
 4            0      137             40             35      168  43.1   
 
    DiabetesPedigreeFunction  Age  Outcome  
 0                     0.627   50        1  
 1                     0.351   31        0  
 2                     0.672   32        1  
 3                     0.167   21        0  
 4                     2.288   33        1  ,
    gender   age  hypertension  heart_disease smoking_history    bmi  \
 0  Female  80.0             0              1           never  25.19   
 1  Female  54.0             0              0         No Info  27.32   
 2    Male  28.0             0              0           never  27.32   
 

### 2. Clean Data and Align Columns

In [3]:
# Fill missing columns in the first dataset
for col in ["gender", "hypertension", "heart_disease", "smoking_history", "HbA1c_level"]:
    if col != "smoking_history":
        first_dataset[col] = 0
    else:
        first_dataset[col] = "never"
first_dataset["blood_glucose_level"] = first_dataset["Glucose"]

# Fill missing columns in the second dataset
for col in ["Pregnancies", "BloodPressure", "SkinThickness", "Insulin",
            "DiabetesPedigreeFunction", "Glucose"]:
    second_dataset[col] = 0
second_dataset["Glucose"] = second_dataset["blood_glucose_level"]

# Concatenate datasets row-wise
combined_df = pd.concat([first_dataset, second_dataset], ignore_index=True)
combined_df.shape

(100768, 18)

### 3. Shuffle and Split Data

In [4]:
np.random.seed(42)
indices = np.arange(len(combined_df))
np.random.shuffle(indices)
split_idx = int(len(combined_df) * 0.8)  # 80% train, 20% val
train_df = combined_df.iloc[indices[:split_idx]].copy()
val_df   = combined_df.iloc[indices[split_idx:]].copy()

### 4. Convert Zero Values to Median Values
*For preservation of large datasets*

In [5]:
zero_as_missing = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI",
                   "HbA1c_level", "blood_glucose_level"]

# Replace zeros with NaN
for col in zero_as_missing:
    train_df[col].replace(0, np.nan, inplace=True)
    val_df[col].replace(0, np.nan, inplace=True)

# Compute medians on training only
medians = train_df[zero_as_missing].median()

# Fill NaNs
train_df[zero_as_missing] = train_df[zero_as_missing].fillna(medians)
val_df[zero_as_missing]   = val_df[zero_as_missing].fillna(medians)

### 5. Encode Categorical Data to Continuous Data

In [6]:
# Gender: Male=1, Female=0
train_df['gender'] = train_df['gender'].map({'Male': 1, 'Female': 0}).fillna(0)
val_df['gender']   = val_df['gender'].map({'Male': 1, 'Female': 0}).fillna(0)

# Smoking history: replace 'No Info' with 'never'
train_df['smoking_history'] = train_df['smoking_history'].replace('No Info', 'never')
val_df['smoking_history']   = val_df['smoking_history'].replace('No Info', 'never')

# One-hot encode smoking history
train_smoke = pd.get_dummies(train_df['smoking_history'], prefix='smoking_history', drop_first=False)
val_smoke   = pd.get_dummies(val_df['smoking_history'], prefix='smoking_history', drop_first=False)

# Ensure all dummy columns exist
expected_smoking_cols = ['smoking_history_never', 'smoking_history_former', 'smoking_history_current']
train_smoke = train_smoke.reindex(columns=expected_smoking_cols, fill_value=0)
val_smoke   = val_smoke.reindex(columns=expected_smoking_cols, fill_value=0)

# Add back to original DataFrames
train_df = pd.concat([train_df, train_smoke], axis=1)
val_df   = pd.concat([val_df, val_smoke], axis=1)

# Fill any remaining NaNs with 0
train_df = train_df.fillna(0)
val_df   = val_df.fillna(0)

train_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,gender,...,heart_disease,smoking_history,HbA1c_level,blood_glucose_level,age,bmi,diabetes,smoking_history_never,smoking_history_former,smoking_history_current
77687,0,126.0,72.0,29.0,125.0,32.4,0.0,0.0,0.0,0.0,...,0,never,4.5,126.0,39.0,27.32,0.0,True,False,False
30880,0,159.0,72.0,29.0,125.0,32.4,0.0,0.0,0.0,0.0,...,0,never,5.7,159.0,58.0,33.83,1.0,True,False,False
99917,0,280.0,72.0,29.0,125.0,32.4,0.0,0.0,0.0,0.0,...,0,ever,5.7,280.0,54.0,31.86,1.0,False,False,False
44947,0,145.0,72.0,29.0,125.0,32.4,0.0,0.0,0.0,0.0,...,0,former,6.0,145.0,70.0,32.40,0.0,False,True,False
38535,0,126.0,72.0,29.0,125.0,32.4,0.0,0.0,0.0,0.0,...,0,never,5.7,126.0,33.0,22.40,0.0,True,False,False


### 6. Final Feature List

In [7]:
numeric_cols = [
    "Pregnancies", "blood_glucose_level", "BloodPressure", "SkinThickness",
    "Insulin", "BMI", "DiabetesPedigreeFunction", "Age", "HbA1c_level"
]

categorical_cols = ["gender", "hypertension", "heart_disease"]

smoking_cols = [
    "smoking_history_never",
    "smoking_history_former",
    "smoking_history_current"
]

feature_cols = numeric_cols + categorical_cols + smoking_cols
print("Total features:", len(feature_cols))

Total features: 15


### 7. Extract X and y Values (Parameters and Validation Outputs)

In [8]:
X_train = train_df[feature_cols].astype(np.float32)
y_train = train_df.get("Outcome", train_df.get("diabetes", pd.Series(0))).astype(int)

X_val = val_df[feature_cols].astype(np.float32)
y_val = val_df.get("Outcome", val_df.get("diabetes", pd.Series(0))).astype(int)

# Ensure no NaNs or Infs
X_train = X_train.fillna(0).replace([np.inf, -np.inf], 0)
X_val   = X_val.fillna(0).replace([np.inf, -np.inf], 0)

X_val[:5], y_val[:5]

(       Pregnancies  blood_glucose_level  BloodPressure  SkinThickness  \
 67078          0.0                140.0           72.0           29.0   
 74397          0.0                280.0           72.0           29.0   
 45823          0.0                 85.0           72.0           29.0   
 44830          0.0                260.0           72.0           29.0   
 19856          0.0                155.0           72.0           29.0   
 
        Insulin        BMI  DiabetesPedigreeFunction  Age  HbA1c_level  gender  \
 67078    125.0  32.400002                       0.0  0.0          5.8     0.0   
 74397    125.0  32.400002                       0.0  0.0          6.5     0.0   
 45823    125.0  32.400002                       0.0  0.0          6.6     0.0   
 44830    125.0  32.400002                       0.0  0.0          6.5     0.0   
 19856    125.0  32.400002                       0.0  0.0          5.8     0.0   
 
        hypertension  heart_disease  smoking_history_never  

### 8. Standardize Mean and Standard Deviation Data

In [9]:
mean = X_train[numeric_cols].mean()
std  = X_train[numeric_cols].std()
std[std == 0] = 1

# Apply scaling
X_train[numeric_cols] = (X_train[numeric_cols] - mean) / std
X_val[numeric_cols]   = (X_val[numeric_cols] - mean) / std

### 9. Convert Data to PyTorch Tensors

In [10]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
val_loader   = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=32, shuffle=False)

print("Training loader batches:", len(train_loader))
print("Validation loader batches:", len(val_loader))

Training loader batches: 2520
Validation loader batches: 630


---
## Import Diabetes Model

In [11]:
from src.model.model import DiabetesModel
model = DiabetesModel(input_size=X_val.shape[1])
summary(model, input_size=(1, X_val.shape[1]))

Layer (type:depth-idx)                   Output Shape              Param #
DiabetesModel                            [1, 2]                    --
├─Sequential: 1-1                        [1, 2]                    --
│    └─Linear: 2-1                       [1, 128]                  2,048
│    └─ReLU: 2-2                         [1, 128]                  --
│    └─Linear: 2-3                       [1, 64]                   8,256
│    └─ReLU: 2-4                         [1, 64]                   --
│    └─Dropout: 2-5                      [1, 64]                   --
│    └─Linear: 2-6                       [1, 32]                   2,080
│    └─ReLU: 2-7                         [1, 32]                   --
│    └─Linear: 2-8                       [1, 2]                    66
Total params: 12,450
Trainable params: 12,450
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.05
Estimated Total Size (M

---
## Training the Model

In [12]:
# Enable early stopping if val_loss doesn't improve in 5 epochs
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    mode="min"
)

trainer = L.Trainer(
    max_epochs=20,
    accelerator="auto",
    devices=1,
    log_every_n_steps=10,
    callbacks=[early_stop]
)

trainer.fit(model, train_loader, val_loader)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name      | Type               | Params | Mode  | FLOPs
-----------------------------------------------------------------
0 | net       | Sequential         | 12.5 K | train | 0    
1 | train_acc | MulticlassAccuracy | 0      | train | 0    
2 | val_acc   | MulticlassAccuracy | 0      | train | 0    
-----------------------------------------------------------------
12.5 K    Trainable params
0         Non-trainable params
12.5 K    Total params
0.050     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode
0         Total Flops


Epoch 7: 100%|██████████| 2520/2520 [00:22<00:00, 111.96it/s, v_num=12, train_loss=5.53e-5, train_acc=1.000, val_loss=0.00471, val_acc=0.997] 


### Final Evaluation

In [ ]:
model.eval()  

# Use DataLoader for batch-wise evaluation
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in val_loader:
        logits = model(X_batch)            
        preds = torch.argmax(logits, dim=1)  
        all_preds.append(preds)
        all_labels.append(y_batch)

# Concatenate all batches
all_preds = torch.cat(all_preds)
all_labels = torch.cat(all_labels)

# Compute accuracy
accuracy = (all_preds == all_labels).float().mean().item()
print(f"Final Validation Accuracy: {accuracy:.4f}")

Final Validation Accuracy: 0.9971
